## 1. Serrings

### 1) Import required libraries

In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [9]:
import unidecode
import string
import random
import re
import time, math

### 2) Hyperparameter

In [10]:
num_epochs = 10000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

## 2. Data

### 1) Prepare characters

In [11]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


### 2) Get text data

In [12]:
file = unidecode.unidecode(open('./data/shakespeare.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394



## 3. Functions for text preocessing
### 1) Random Chunk

In [13]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

 regal title and the seat
Of England's true-anointed lawful king.
This is the cause that I, poor Margaret,
With this my son, Prince Edward, Henry's heir,
Am come to crave thy just and lawful aid;
And i


### 2) Character to tensor

In [14]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)#.cuda()

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


### 3) Chunk into input & label

In [15]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

## 3. Model & Optimizer

### 1) Model

In [16]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden,cell):
        out = self.encoder(input.view(batch_size,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden,cell

    def init_hidden(self):
          
        hidden = Variable(torch.zeros(num_layers,batch_size,hidden_size))
        cell = Variable(torch.zeros(num_layers,batch_size,hidden_size))
        
        return hidden,cell
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)

In [17]:
inp = char_tensor("A")
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())

out,hidden,cell = model(inp,hidden,cell)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 100])


### 2) Loss & Optimizer

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

### 3) Test Function

In [18]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden(1)
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

## 4. Train

In [14]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden,cell = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j]
        y,hidden,cell = model(x,hidden,cell)
        loss += loss_func(y,y_.view(-1))

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 tensor(4.5815, grad_fn=<DivBackward0>) 

\{M&b}scRSYE'g[cy9IW&U8'J-)wv(?o-+9;%|b>-%G]!}y.ei@^K C\-X3"U-[>Siw*bu:na\A3MM)
\W]q7WFJ9%OLe}"(1~khh$Pc76&:kQ,WJx@:@ujO#?I 1\{B!P.["kJ5)P)t;Gltw#oG%



 tensor(2.3908, grad_fn=<DivBackward0>) 

b*ged of ars and,
I anke?;
AU:
T
LI:
OOEOFENy amand the onoug at my me he and ga than ris be whouverlrat nos mrrere chof me, hed todcy the kisard be mistte ki ute:
I ane ban ctoutes ires ot the bere se



 tensor(2.3422, grad_fn=<DivBackward0>) 

bd lis.

LIY
I yove the ous ondend lheur ally wone my you arerut ind mh.
Shist the thino nos davel.

NOAELIO: I wot deent;
Beve sist ufreds you swhlis you the dand theve't word yoay sese the an, beal f



 tensor(2.1592, grad_fn=<DivBackward0>) 

bHee.

OUSIANRONG EP:
Mates.
Bat kneet ald tet a sheso
Hele wous-mour, a by pet kind me'd hord that the you tor'y mers the moct hef cofser thig thes ith toaldgeest you so for mory bre she'r ty, eames, 



 tensor(2.1122, grad_fn=<DivBackward0>) 

bard your frokn; bo

That I do my anges, contor and there gord,
To strancty say, iffrock, stroust is here,
Vain my the play my worlds of stake wish restel--
And pla



 tensor(1.6810, grad_fn=<DivBackward0>) 

bed thou statest mine, sany woold loud
Perame, give kind hall dayer,
The looks lord: in is my well in the curnce not shan;
Much, the served let uch to there you broved chate cowarly love is bid he bend



 tensor(1.4092, grad_fn=<DivBackward0>) 

both all shall I womer and not;
And will and hath grose, with you love!

DUKE VINCENTIO:
Hew's thus of his meart you conculk will son sople,
Wourse soul am wounten he mounder
My die bodst the whom thin



 tensor(1.5074, grad_fn=<DivBackward0>) 

blick'd.
'Ford goods the bring-and thinks reast 'tis that lobester'd but the hall be that the gring my hoarer
Prows and speak I hards; I mad?

BRUTUS:
Come, this brance, me frious, I would at sif:
Of t



 tensor(1.5668, grad_fn=<DivBackward0>) 

by you
seek hands on them if my see this friess make to chons the fisi

Tutweres the stay abambrest hin the read, the have be it lear men
His macting my rangman to our 



 tensor(1.6960, grad_fn=<DivBackward0>) 

beach to make a milled send
For nor the throngs to me in a buke is burn a creaded
you seep the not so thy grating a fairing which againd of the late my so ins
When yet of sheed be with of them the so o



 tensor(1.5257, grad_fn=<DivBackward0>) 

brean teast?

KING RICHARD III:
Hath soul the now or hus? I must with that wars' and pasties we hate,
but many we seaven, or others
Here that I have be use, what lood him not wroger
And her fail to mai



 tensor(1.5698, grad_fn=<DivBackward0>) 

by, I hust than his,
I have come the confir, but thy forthing
That play, I'll and ever these at to the fooes,
Marger I must.

Second Seand:
She tare.
I have any be for which and that York
Mast that hav



 tensor(1.5236, grad_fn=<DivBackward0>) 

bery of and stonce,
And peart crienion thou a by as equrse arm
And lage sin this sturked.

HENRY BOLINCE:
O her beath

Be thought stand he shopes of you, see beace, fair 


